In [27]:
from scipy import constants

from PhysicalUnits import PhysicalUnits

tip_radius = 100.0*constants.nano
freq = 50.0*constants.tera
length_FD = 2.0*tip_radius

units = PhysicalUnits(length_FD)
freq_FD = units.ConvertSIFrequencyToFDUnits(freq)
tip_radius_FD = units.ConvertSILengthToFDUnits(tip_radius)
lambda_si = constants.c/freq

print("tip_radius_FD :", tip_radius_FD)
print("freq_FD: ", freq_FD)
print("lambda_si: ", lambda_si)

tip_radius_FD : 0.5
freq_FD:  0.03335640951981521
lambda_si:  5.99584916e-06


In [30]:
import numpy as np

from GridsParameters import MultilevelGridsParameters

box_0_scale = 8.0*tip_radius_FD

mlGrid = MultilevelGridsParameters()
mlGrid.SetCenterGridDimensions(r0 = np.array([-1.0, -2.5, -1.0])*box_0_scale,
                               r1 = np.array([+1.0, +1.0, +1.0])*box_0_scale,
                               dr = np.array([1.0, 1.0, 1.0])*0.1*tip_radius_FD,
                               S  = 0.98,
                               nFactor = 8,
                               nIsPowerOf2=False)

mlGrid.AddLayer(numOfCells = np.array([32, 32, 32]))
mlGrid.AddLayer(numOfCells = np.array([32, 32, 32]))
#mlGrid.AddLayer(numOfCells = np.array([32, 32, 32]))
#mlGrid.AddLayer(numOfCells = np.array([32, 32, 32]))
#mlGrid.AddLayer(numOfCells = np.array([32, 32, 32]))
#mlGrid.AddLayer(numOfCells = np.array([32, 32, 32]))
#mlGrid.AddLayer(numOfCells = np.array([32, 32, 32]))
add_pml = False
if add_pml:
    mlGrid.AddPML(face = "f", numOfCells = 20)
    mlGrid.AddPML(face = "b", numOfCells = 20)
    mlGrid.AddPML(face = "r", numOfCells = 20)
    mlGrid.AddPML(face = "l", numOfCells = 20)

sourceType = "tip"
if sourceType == "pointSource":
    mlGrid.AddSource({"type": "GaussianPointSource", "position":np.array([0.0, 0.0, 0.0]), 
                      "polarization":"y", "amplitude":1.0, "t_center":1.0/freq_FD, "t_decay":0.25/freq_FD,
                      "modulationFrequency":freq_FD , "modulationPhase":np.pi/2 , "timeOffsetFraction":0.5
                     })
elif sourceType == "tip":
    mlGrid.AddGeometry({"type":"cone", "geometryName":"tip",
                        "coneAngle":1.0*(np.pi/180), 
                        "tipRadius":tip_radius_FD, "apexPosition":np.array([0.0, 0.0, 0.0])*box_0_scale,
                        "height":2.0*box_0_scale
                      })
    use_pec = True
    if use_pec:
        mlGrid.AddMaterial({"type":"pec_PureScattered", "geometryName":"tip"})
    else:
        mlGrid.AddMaterial({"type":"DrudeMetal_PureScattered", "geometryName":"tip",
                            "plasmaFrequency":2.0*np.pi*1, "scatteringRate":2.0*np.pi*1})
        
    mlGrid.AddSource({"type": "PureScatteredRectPlaneWave", "geometryName":"tip",
                      "polarization":"y", "amplitude":1.0, 
                      "propagationDirection":np.array([0,0,1.0]), "propagationVelocity":1.0,
                      "t_center":0.6/freq_FD, "rectWidth":1.0/freq_FD, "rectEdgeWidth":0.1/freq_FD,
                      "modulationFrequency":freq_FD , "modulationPhase":np.pi/2 
                 })


mlGrid.AddView({"type":"partial", "plane":"x", "at":0.0, "direction":"y", "arrayName":"E"})
mlGrid.AddView({"type":"partial", "plane":"y", "at":0.0, "direction":"y", "arrayName":"E"})
mlGrid.AddView({"type":"partial", "plane":"z", "at":0.0, "direction":"y", "arrayName":"E"})

paramfileName = "../../../data/3D/auto/params.param"
mlGrid.SetupCollectionAndRun(t_max = 1.2/freq_FD, 
                             filename = "../../processed/Maxwell3D_nonuniform_autogenerated.json", 
                             paramfileName = paramfileName
                             )

import pickle
fileParams = open(paramfileName, "rb")
params = pickle.load(fileParams)
fileParams.close()

params["units"] = {"tip_radius":tip_radius, "freq":freq, "length_FD":length_FD}

paramfile = open(paramfileName, "wb")
pickle.dump(params, paramfile)
paramfile.close()

(params["boxes"][-1]["r1"] - params["boxes"][-1]["r0"])*length_FD/lambda_si

Num of time steps:  317
gridsIntersectingGeometries  {'grid_m0': {'tip': [array([-0.56547055, -8.        , -0.56547055]), array([0.56547055, 0.        , 0.56547055])]}, 'grid_r1': {}, 'grid_l1': {}, 'grid_u1': {}, 'grid_d1': {}, 'grid_f1': {}, 'grid_b1': {}, 'grid_r2': {}, 'grid_l2': {}, 'grid_u2': {}, 'grid_d2': {}, 'grid_f2': {}, 'grid_b2': {}}


array([0.96066459, 1.17414562, 0.96066459])